In [ ]:
#imports, global consts, inits
import ee
import geemap
#import calendar
from datetime import datetime

NDVI_VIZ_PARAMS = {"min" : -1, "max": 1, "palette" : ["blue", "white", "green"]}
NDVI_VIZ_PARAMS_CROP_ONLY = {"min" : 0.1, "max": 0.5, "palette" : ["red", "yellow", "green"]}


ee.Initialize(project='seamproj01')
map = geemap.Map()

In [ ]:
def AddNDVI(col : ee.ImageCollection) -> ee.ImageCollection:
    def NDVIize(img : ee.Image) -> ee.Image:
        return img.addBands(img.normalizedDifference(["sur_refl_b02", "sur_refl_b01"]).rename("NDVI"))
    return col.map(NDVIize)


def GetQualityMask(img : ee.Image) -> ee.Image:
    def bitwiseExtract(value, fromBit, toBit = None):
    #https://gis.stackexchange.com/questions/349371/creating-cloud-free-images-out-of-a-mod09a1-modis-image-in-gee
        if (toBit == None):
             toBit = fromBit
        maskSize = ee.Number(1).add(toBit).subtract(fromBit)
        mask = ee.Number(1).leftShift(maskSize).subtract(1)
        return value.rightShift(fromBit).bitwiseAnd(mask)
    
    state = img.select("State")

    stateCloud = bitwiseExtract(state, 0, 1)
    stateCloudShadow = bitwiseExtract(state, 2)
    stateLC = bitwiseExtract(state, 3, 5)
    stateCirrus = bitwiseExtract(state, 8, 9)
    
    mask = stateCloud.eq(0).Or(stateCloud.eq(3)) #clear pixel or unset cloud state (assumed clear)
    mask = mask.And(stateCloudShadow.eq(0)) #no cloud shadow
    mask = mask.And(stateLC.eq(1)) #land
    mask = mask.And(stateCirrus.eq(0).Or(stateCirrus.eq(1)))
    return mask


def MaskOutPoorQualityPixels(col : ee.ImageCollection) -> ee.ImageCollection:
    def UpdateMask(img : ee.Image) -> ee.Image:
        mask = GetQualityMask(img)
        return img.updateMask(mask)
    
    return col.map(UpdateMask)


def GetDatesInCollection(collection : ee.imagecollection, filterDuplicates : bool = True) -> list[str]:
    datesInRange = collection
    
    if filterDuplicates:
        datesInRange = collection.distinct("system:time_start")

    datesInRange = datesInRange.aggregate_array('system:time_start')
    #datesInRange = datesInRange.map(lambda x : x)
    dates = datesInRange.getInfo()
    
    for i in range(0, len(dates)):
        dates[i] = datetime.datetime.fromtimestamp(dates[i]).strftime('%c')
    
    dates.sort()
    return dates


In [ ]:
sudanNationalBorders = ee.FeatureCollection("projects/seamproj01/assets/SudanBorders") #uploaded via the web interface
sudanStateBorders = ee.FeatureCollection("projects/seamproj01/assets/SudanStateBorders") #uploaded via the web interface
sudanCroplandMask = ee.FeatureCollection("projects/seamproj01/assets/Sudan_Cropland_Mask_CopernicusLCLU2019") #uploaded via the web interface
#geziraCroplandMask = ee.FeatureCollection("projects/seamproj01/assets/Gezira_Cropland_Mask_CopernicusLCLU2019")

testArea =  ee.FeatureCollection("projects/seamproj01/assets/test_area").geometry()
khartoum = sudanStateBorders.filter(ee.Filter.inList("ADM1_EN", rightValue=["Khartoum"])).geometry()
gezira = sudanStateBorders.filter(ee.Filter.inList("ADM1_EN", rightValue=["Aj Jazirah"])).geometry()

khartoumCropland = sudanCroplandMask.filter(ee.Filter.inList("ADM1_EN", rightValue=["Khartoum"])).geometry()
geziraCropland = sudanCroplandMask.filter(ee.Filter.inList("ADM1_EN", rightValue=["Aj Jazirah"])).geometry()
khartoumCroplandModified = ee.FeatureCollection("projects/seamproj01/assets/khartoum_cropmask_gt_1km2").geometry()
geziraCroplandModified = ee.FeatureCollection("projects/seamproj01/assets/gezira_cropmask_western_bank").geometry()

roi = khartoum
roiCropMask = khartoumCroplandModified

#time-series limits
yearStart = 2018
monthStart = 1
yearEnd = 2023 #inclusive
monthEnd = 12 #inclusive

#months comprising the season for analysis. MUST BE 4!!!
targetMonths = [7, 8, 9, 10]

dateStart = str(yearStart)+ "-" + str(monthStart) + "-1"
dateEnd = str(yearEnd) + '-' + str(monthEnd + 1) + '-1' if monthEnd < 12 else str(yearEnd + 1) + '-1-1'

modis = ee.ImageCollection("MODIS/061/MOD09Q1")
modis = modis.filterBounds(roi).filterDate(dateStart, dateEnd)
modis = AddNDVI(MaskOutPoorQualityPixels(modis)).map(lambda img : img.select("NDVI").clip(roiCropMask))


In [ ]:
timeSeries = ee.List([])

for year in range(yearStart, yearEnd + 1):
    yearTS = ee.List([])
    #for month in range(1, 13):
    for month in targetMonths:
        if ((year * 100) + month > (yearEnd * 100) + monthEnd):
            break

        subPeriodStart = str(year) + '-' + str(month) + '-1'
        subPeriodEnd = str(year) + '-' + str(month + 1) + '-1' if month < 12 else str(year + 1) + '-1-1'
        subCol = modis.filterDate(subPeriodStart, subPeriodEnd)
        
        minMaxNDVI = subCol.reduce(ee.Reducer.minMax())

        monthMax = minMaxNDVI.select("NDVI_max").rename("max")
        monthRange = minMaxNDVI.select("NDVI_max").subtract(minMaxNDVI.select("NDVI_min")).rename("range")
        yearTS = yearTS.add(ee.Image([monthMax, monthRange]).set({"system:index" : "month_" + str(month), "system:id" : "month_" + str(month)}))
    
    timeSeries = timeSeries.add(ee.ImageCollection(yearTS).toBands().set({"year" : year}))

timeSeries = ee.ImageCollection(timeSeries)

monthlyMedianNDVI = ee.List([])
pureCropNDVI = ee.List([])

#for month in range(1, 13):
for month in targetMonths:
    prefix = "month_" + str(month) + "_"
    thisMonthTS = timeSeries.select([prefix + "max", prefix + "range"])

    monthMedian = thisMonthTS.reduce(ee.Reducer.median()).rename([prefix + "max", prefix + "range"]).set({"month" : month})

    monthlyMedianNDVI = monthlyMedianNDVI.add(monthMedian)
    
    meanStdReducer = ee.Reducer.mean().combine(ee.Reducer.stdDev(), sharedInputs = True)

    monthPureCropNDVI = thisMonthTS.map(lambda img : img.updateMask(img.gt(monthMedian)))
    monthPureCropNDVI = monthPureCropNDVI.reduce(meanStdReducer).set("month", month)

    pureCropNDVI = pureCropNDVI.add(monthPureCropNDVI)

monthlyMedianNDVI = ee.ImageCollection(monthlyMedianNDVI)
pureCropNDVI = ee.ImageCollection(pureCropNDVI)

In [ ]:
temporalAnomalies = ee.List([])

for year in range (yearStart, yearEnd + 1):
    seasonAnomalies = ee.List([])

    #for month in range(1, 13):
    for month in targetMonths:
        if ((year * 100) + month > (yearEnd * 100) + monthEnd):
            break

        prefix = "month_" + str(month) + "_"
        pureCropSignal = pureCropNDVI.filter(ee.Filter.eq("month", month)).first()
        
        ta = timeSeries.filter(ee.Filter.eq("year", year)).first()

        taMax = ta.select(prefix + "max").subtract(pureCropSignal.select(prefix + "max_mean")).divide(pureCropSignal.select(prefix + "max_stdDev")).rename("max")
        taRange = ta.select(prefix + "range").subtract(pureCropSignal.select(prefix + "range_mean")).divide(pureCropSignal.select(prefix + "range_stdDev")).rename("range")

        ta = ee.Image([taMax, taRange]).set({"system:index" : "month_" + str(month), "system:id" : "month_" + str(month)})

        seasonAnomalies = seasonAnomalies.add(ta)

    seasonAnomalies = ee.ImageCollection(seasonAnomalies).toBands().set({"year" : year})
    temporalAnomalies = temporalAnomalies.add(seasonAnomalies)

temporalAnomalies = ee.ImageCollection(temporalAnomalies)

In [ ]:
#Temporal analysis component

def TemporalAnomalyAnalysis(image : ee.Image) -> ee.Image: #to be mapped over temporalAnomalies collection
    taMax = []
    taRange = []

    # for i in range (0, 4):
    #     name = "month_" + str(i)
    #     taMax.append(image.select("month_" + str(targetMonths[i]) + "_max").rename(name))
    #     taRange.append(image.select("month_" + str(targetMonths[i]) + "_range").rename(name))

    for month in targetMonths:
        name = "month_" + str(month)
        taMax.append(image.select(name + "_max").rename(name))
        taRange.append(image.select(name + "_range").rename(name))
        
    isFallow_1 = taMax[0].lt(-3).And(taMax[1].lt(-3)).And(taMax[2].lt(-3))
    isFallow_1 = isFallow_1.Or(taMax[1].lt(-3).And(taMax[2].lt(-3)).And(taMax[3].lt(-3)))
    isFallow_1 = isFallow_1.rename("max")

    isFallow_2 = taRange[0].lt(-3).And(taRange[1].lt(-3)).And(taRange[2].lt(-3))
    isFallow_2 = isFallow_2.Or(taRange[1].lt(-3).And(taRange[2].lt(-3)).And(taRange[3].lt(-3)))
    isFallow_2 = isFallow_2.rename("range")

    return ee.Image([isFallow_1, isFallow_2]).set({"year" : image.get("year")})

#isFallow_TA the two questions for the temporal anomalies
isFallow_TA = temporalAnomalies.map(TemporalAnomalyAnalysis)

In [ ]:
def SpatialAnomalyAnalysis(image : ee.Image) -> ee.Image:
    #image has max and range for each month in a year (12 bands). month_x_max, month_x_range, x = month number
    maxOfMax = ee.List([])
    maxOfRange = ee.List([])
    
    spatialMedianMax = ee.List([])
    spatialMedianRange = ee.List([])

    # for i in range (0, 4):
    #     name = "month_" + str(i)
    #     _maxOfMax = image.select("month_" + str(targetMonths[i]) + "_max").rename("max")
    #     _maxOfRange = image.select("month_" + str(targetMonths[i]) + "_range").rename("range")

    for month in targetMonths:
        name = "month_" + str(month)
        _maxOfMax = image.select(name + "_max").rename("max")
        _maxOfRange = image.select(name + "_range").rename("range")
        
        maxOfMax = maxOfMax.add(_maxOfMax)
        maxOfRange = maxOfRange.add(_maxOfRange)


        spatialMedianMax =      spatialMedianMax.add(   _maxOfMax.reduceRegion(
                                                        reducer = ee.Reducer.median(),
                                                        geometry = roiCropMask,
                                                        tileScale = 4,
                                                        scale = 250,
                                                        crs='EPSG:4326',
                                                        ).getNumber("max"))
        
        spatialMedianRange =   spatialMedianRange.add(  _maxOfRange.reduceRegion(
                                                        reducer = ee.Reducer.median(),
                                                        geometry = roiCropMask,
                                                        tileScale = 4,
                                                        scale = 250,
                                                        crs='EPSG:4326',
                                                        ).getNumber("range"))


    maxOfMax = ee.ImageCollection(maxOfMax)
    maxOfRange = ee.ImageCollection(maxOfRange)

    spatialMedianMax = ee.Number(spatialMedianMax.reduce(ee.Reducer.max()))
    spatialMedianRange = ee.Number(spatialMedianRange.reduce(ee.Reducer.max()))

    isFallow_3 = maxOfMax.reduce(ee.Reducer.max()).lt(ee.Number(0.8).multiply(spatialMedianMax)).set({"year" : year}).rename("max")
    isFallow_4 = maxOfRange.reduce(ee.Reducer.max()).lt(ee.Number(0.8).multiply(spatialMedianRange)).set({"year" : year}).rename("range")

    return ee.Image([isFallow_3, isFallow_4]).set({"year" : image.get("year")})

#isFallow_SA the two questions for the spatial anomalies
isFallow_SA = timeSeries.map(SpatialAnomalyAnalysis)

In [ ]:
# #test
# spatialMedianMax = ee.List([])

# for i in range (0, 4):
#     _maxOfMax = timeSeries.filter(ee.Filter.eq("year", 2010)).first().select("month_" + str(targetMonths[i]) + "_max").rename("max")

#     spatialMedianMax =      spatialMedianMax.add(   _maxOfMax.reduceRegion(
#                                                         reducer = ee.Reducer.median(),
#                                                         geometry = roiCropMask,
#                                                         #geometry = roi,
#                                                         tileScale = 4,
#                                                         scale = 250,
#                                                         crs='EPSG:4326',
#                                                         ).getNumber("max"))


# print(spatialMedianMax.getInfo())
# spatialMedianMax = ee.Number(spatialMedianMax.reduce(ee.Reducer.max()))
# print(spatialMedianMax.getInfo())

# print (ee.Number(0.8).multiply(spatialMedianMax).getInfo())

# # [0.1600498922929848, 0.20697533763447257, 0.261636471000723, 0.2438680491313201]
# # 0.261636471000723

In [ ]:
#Final analysis

fallowTS = ee.List([])

scoreIsFallow_TA = isFallow_TA.map(lambda img : img.reduce(ee.Reducer.sum()).set({"year" : img.get("year")}).rename("score"))
scoreIsFallow_SA = isFallow_SA.map(lambda img : img.reduce(ee.Reducer.sum()).set({"year" : img.get("year")}).rename("score"))

for year in range(yearStart, yearEnd + 1):
    isFallow = scoreIsFallow_SA.filter(ee.Filter.eq("year", year)).first()
    isFallow = isFallow.add(scoreIsFallow_TA.filter(ee.Filter.eq("year", year)).first())

    isFallow = isFallow.gte(ee.Number(2)).rename(str(year)).set({"year" : year})
    fallowTS = fallowTS.add(isFallow)

fallowTS = ee.ImageCollection(fallowTS).toBands()

In [16]:
# #export
noDataValue = -9999

task = ee.batch.Export.image.toDrive(
    image=fallowTS,
    description='fallow_ts_' + str(yearStart) + '-' + str(yearEnd),
    #folder='ee_export',
    region=roi,
    scale=250,
    crs='EPSG:4326',
    fileFormat='GeoTIFF',
    formatOptions={
        'noData': noDataValue
    })

task.start()


isFallowComponents = ee.List([])
for year in range (yearStart, yearEnd + 1):

    yearComponents = isFallow_TA.filter(ee.Filter.eq("year", year)).first().rename([str(year) + "_ta_max", str(year) + "_ta_range"])
    yearComponents = yearComponents.addBands(isFallow_SA.filter(ee.Filter.eq("year", year)).first().rename([str(year) + "_sa_max", str(year) + "_sa_range"]))
    isFallowComponents = isFallowComponents.add(yearComponents)

isFallowComponents = ee.ImageCollection(isFallowComponents).toBands()

task = ee.batch.Export.image.toDrive(
image=isFallowComponents,
description='fallow_components_'  + str(yearStart) + '-' + str(yearEnd),
#folder='ee_export',
region=roi,
scale=250,
crs='EPSG:4326',
fileFormat='GeoTIFF',
formatOptions={
    'noData': noDataValue
})

task.start()


In [17]:
# map.centerObject(roi)
# map.addLayer(fallowTS, name="FallowTS")
# map